## 1 - Importar Pacotes

In [29]:
import pandas as pd

## 2 - Importar Dados

In [30]:
# Importar o arquivo e já converter as colunas de de data/tempo de string para date_time
df = pd.read_csv("Dataset_publico.csv", sep=';', parse_dates=['TS_INC_EPR','TS_INC_CHMD'])
df.head()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD
0,0,0,2020-09-01 10:01:24,2020-09-01 10:02:13
1,1,0,2020-09-01 10:04:21,2020-09-01 10:22:01
2,2,0,2020-09-01 10:16:14,2020-09-01 10:25:06
3,3,0,2020-09-01 10:28:38,2020-09-01 10:28:40
4,4,0,2020-09-01 10:37:38,2020-09-01 10:37:41


In [31]:
# Verificar o tamanho da amostra
df.shape

(335612, 4)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335612 entries, 0 to 335611
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   ID           335612 non-null  int64         
 1   ID_UOR       335612 non-null  int64         
 2   TS_INC_EPR   335612 non-null  datetime64[ns]
 3   TS_INC_CHMD  335428 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2)
memory usage: 10.2 MB


## 3 - Tratar valores nulos

In [33]:
df[df['TS_INC_CHMD'].isna()]

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD
10228,10228,0,2021-02-24 15:47:22,NaT
11844,11844,0,2021-03-24 15:28:14,NaT
19149,21141,1,2020-09-23 14:55:42,NaT
22495,24487,1,2020-11-12 15:00:58,NaT
27841,29833,1,2021-01-21 15:13:14,NaT
...,...,...,...,...
332793,373863,24,2021-01-06 13:45:31,NaT
332979,374049,24,2021-01-19 14:40:00,NaT
333768,374838,24,2021-03-18 15:06:22,NaT
334469,375539,24,2021-05-11 14:20:22,NaT


In [34]:
# Excluir
df.dropna(inplace=True)
df.shape

(335428, 4)

## 4 - Feature Engenering

### 4.1 - Quanto tempo durou cada atendimento?

In [35]:
# Subtrair o tempo final pelo inicial
df['tempo_de_espera'] = df['TS_INC_CHMD'] - df['TS_INC_EPR']

# Extrair os minutos do tempo de espera
df['total_min_espera'] = (df['tempo_de_espera'].dt.total_seconds()//60).astype(int)

df.head()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,tempo_de_espera,total_min_espera
0,0,0,2020-09-01 10:01:24,2020-09-01 10:02:13,0 days 00:00:49,0
1,1,0,2020-09-01 10:04:21,2020-09-01 10:22:01,0 days 00:17:40,17
2,2,0,2020-09-01 10:16:14,2020-09-01 10:25:06,0 days 00:08:52,8
3,3,0,2020-09-01 10:28:38,2020-09-01 10:28:40,0 days 00:00:02,0
4,4,0,2020-09-01 10:37:38,2020-09-01 10:37:41,0 days 00:00:03,0


### 4.2 - Quais atendimentos tiveram tempo de espera superior a 15 minutos?

In [36]:
# Criar coluna clientes atrasados
df['atrasado'] = 0
df.loc[df['tempo_de_espera']> '00:15:00','atrasado'] = 1
df.head()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,tempo_de_espera,total_min_espera,atrasado
0,0,0,2020-09-01 10:01:24,2020-09-01 10:02:13,0 days 00:00:49,0,0
1,1,0,2020-09-01 10:04:21,2020-09-01 10:22:01,0 days 00:17:40,17,1
2,2,0,2020-09-01 10:16:14,2020-09-01 10:25:06,0 days 00:08:52,8,0
3,3,0,2020-09-01 10:28:38,2020-09-01 10:28:40,0 days 00:00:02,0,0
4,4,0,2020-09-01 10:37:38,2020-09-01 10:37:41,0 days 00:00:03,0,0


### 4.3 Quantos atendimentos foram feitos por faixa de horário, por dia, por UOR?

- Protocolos emitidos com TS_INC_EPR antes das 10h, devem ser contabilizados entre 10:00:00 e 10:59:59, mantendo a contabilização total do tempo de espera.
- Os protocolos com horário de emissão (TS_INC_EPR) das 14h em diante, devem ser contabilizados entre 14:00:00 e 14:59:59.

#### a) Criar coluna data e faixa_horario

In [37]:
from pandas.tseries.offsets import *
df['data'] = df['TS_INC_EPR'].dt.date
df['faixa_horario'] = df['TS_INC_EPR'].dt.hour
df.head()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,tempo_de_espera,total_min_espera,atrasado,data,faixa_horario
0,0,0,2020-09-01 10:01:24,2020-09-01 10:02:13,0 days 00:00:49,0,0,2020-09-01,10
1,1,0,2020-09-01 10:04:21,2020-09-01 10:22:01,0 days 00:17:40,17,1,2020-09-01,10
2,2,0,2020-09-01 10:16:14,2020-09-01 10:25:06,0 days 00:08:52,8,0,2020-09-01,10
3,3,0,2020-09-01 10:28:38,2020-09-01 10:28:40,0 days 00:00:02,0,0,2020-09-01,10
4,4,0,2020-09-01 10:37:38,2020-09-01 10:37:41,0 days 00:00:03,0,0,2020-09-01,10


#### b) Agrupar horários fora da faixa

In [38]:
# Verificar as faixas de horário
df.groupby('faixa_horario').count()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,tempo_de_espera,total_min_espera,atrasado,data
faixa_horario,,,,,,,,
7,1,1,1,1,1,1,1,1
8,12,12,12,12,12,12,12,12
9,3700,3700,3700,3700,3700,3700,3700,3700
10,54676,54676,54676,54676,54676,54676,54676,54676
11,74907,74907,74907,74907,74907,74907,74907,74907
12,64149,64149,64149,64149,64149,64149,64149,64149
13,69343,69343,69343,69343,69343,69343,69343,69343
14,41427,41427,41427,41427,41427,41427,41427,41427
15,25144,25144,25144,25144,25144,25144,25144,25144


In [39]:
df_agrupado = df.copy()

# Adequar a faixa abaixo das 10:00
df_agrupado.loc[df_agrupado['faixa_horario']<10,'faixa_horario'] = 10

# Adequar a faixa acima das 14:00
df_agrupado.loc[df_agrupado['faixa_horario']>14, 'faixa_horario'] = 14

In [40]:
# Verificar as alterações
df_agrupado.groupby('faixa_horario').count()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,tempo_de_espera,total_min_espera,atrasado,data
faixa_horario,,,,,,,,
10,58389,58389,58389,58389,58389,58389,58389,58389
11,74907,74907,74907,74907,74907,74907,74907,74907
12,64149,64149,64149,64149,64149,64149,64149,64149
13,69343,69343,69343,69343,69343,69343,69343,69343
14,68640,68640,68640,68640,68640,68640,68640,68640


In [41]:
df_agrupado.head()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,tempo_de_espera,total_min_espera,atrasado,data,faixa_horario
0,0,0,2020-09-01 10:01:24,2020-09-01 10:02:13,0 days 00:00:49,0,0,2020-09-01,10
1,1,0,2020-09-01 10:04:21,2020-09-01 10:22:01,0 days 00:17:40,17,1,2020-09-01,10
2,2,0,2020-09-01 10:16:14,2020-09-01 10:25:06,0 days 00:08:52,8,0,2020-09-01,10
3,3,0,2020-09-01 10:28:38,2020-09-01 10:28:40,0 days 00:00:02,0,0,2020-09-01,10
4,4,0,2020-09-01 10:37:38,2020-09-01 10:37:41,0 days 00:00:03,0,0,2020-09-01,10


In [42]:
# Formatar faixa em data e horário
df_agrupado['faixa_horario'] = df_agrupado['data'].astype('str') + ' ' + (df_agrupado['faixa_horario'].astype('str')) + ":00:00"

In [43]:
df_agrupado.head()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,tempo_de_espera,total_min_espera,atrasado,data,faixa_horario
0,0,0,2020-09-01 10:01:24,2020-09-01 10:02:13,0 days 00:00:49,0,0,2020-09-01,2020-09-01 10:00:00
1,1,0,2020-09-01 10:04:21,2020-09-01 10:22:01,0 days 00:17:40,17,1,2020-09-01,2020-09-01 10:00:00
2,2,0,2020-09-01 10:16:14,2020-09-01 10:25:06,0 days 00:08:52,8,0,2020-09-01,2020-09-01 10:00:00
3,3,0,2020-09-01 10:28:38,2020-09-01 10:28:40,0 days 00:00:02,0,0,2020-09-01,2020-09-01 10:00:00
4,4,0,2020-09-01 10:37:38,2020-09-01 10:37:41,0 days 00:00:03,0,0,2020-09-01,2020-09-01 10:00:00


#### c) Agrupar total de clientes atendidos por data, faixa e ID_UOR

In [44]:
df_agrupado[['data','ID','ID_UOR','faixa_horario', 'atrasado']]

,data,ID,ID_UOR,faixa_horario,atrasado
0,2020-09-01,0,0,2020-09-01 10:00:00,0
1,2020-09-01,1,0,2020-09-01 10:00:00,1
2,2020-09-01,2,0,2020-09-01 10:00:00,0
3,2020-09-01,3,0,2020-09-01 10:00:00,0
4,2020-09-01,4,0,2020-09-01 10:00:00,0
...,...,...,...,...,...
335607,2021-07-30,376677,24,2021-07-30 13:00:00,0
335608,2021-07-30,376678,24,2021-07-30 13:00:00,0
335609,2021-07-30,376679,24,2021-07-30 13:00:00,0
335610,2021-07-30,376680,24,2021-07-30 13:00:00,0


In [47]:
# Agrupar final no total clientes atendidos
total_no_prazo = df_agrupado[df_agrupado['atrasado']==0].groupby(
    ['faixa_horario','ID_UOR'], as_index=False).count()[['faixa_horario','ID_UOR','ID']]
total_no_prazo.rename(columns={'ID': 'total_no_prazo'}, inplace = True)
total_no_prazo

,faixa_horario,ID_UOR,total_no_prazo
0,2020-09-01 10:00:00,0,11
1,2020-09-01 10:00:00,1,28
2,2020-09-01 10:00:00,2,17
3,2020-09-01 10:00:00,4,28
4,2020-09-01 10:00:00,9,22
...,...,...,...
20729,2021-07-30 14:00:00,14,1
20730,2021-07-30 14:00:00,18,3
20731,2021-07-30 14:00:00,21,1
20732,2021-07-30 14:00:00,22,2


In [48]:
# Agrupar final no total por clientes atendidos com mais de  minutos
total_atrasados = df_agrupado[df_agrupado['atrasado']==1].groupby(
    ['faixa_horario','ID_UOR'], as_index=False).count()[['faixa_horario','ID_UOR','ID']]
total_atrasados.rename(columns={'ID': 'total_atrasados'}, inplace = True)
total_atrasados

,faixa_horario,ID_UOR,total_atrasados
0,2020-09-01 10:00:00,0,1
1,2020-09-01 10:00:00,1,4
2,2020-09-01 10:00:00,5,1
3,2020-09-01 11:00:00,1,1
4,2020-09-01 11:00:00,8,10
...,...,...,...
5060,2021-07-30 13:00:00,14,4
5061,2021-07-30 13:00:00,15,2
5062,2021-07-30 13:00:00,19,8
5063,2021-07-30 14:00:00,4,2


In [49]:
# Mesclar "No prazo" + "Atrasados" e gerar a porcentagem
total_por_faixa = total_no_prazo.merge(total_atrasados, how='left', on=['faixa_horario', 'ID_UOR'])
total_por_faixa.fillna(value=0, inplace=True)
total_por_faixa['total_atrasados'] = total_por_faixa['total_atrasados'].astype('int')
total_por_faixa['total_geral'] = total_por_faixa['total_atrasados'] + total_por_faixa['total_no_prazo']
total_por_faixa['porcentagem_atraso'] = round(total_por_faixa['total_atrasados'] / total_por_faixa['total_geral'],2)

In [50]:
total_por_faixa['faixa_horario'] = pd.to_datetime(total_por_faixa['faixa_horario'])

In [51]:
total_por_faixa.set_index('faixa_horario', inplace=True)

In [77]:
total_por_faixa

,ID_UOR,total_no_prazo,total_atrasados,total_geral,porcentagem_atraso
faixa_horario,,,,,
2020-09-01 10:00:00,0,11,1,12,0.08
2020-09-01 10:00:00,1,28,4,32,0.12
2020-09-01 10:00:00,2,17,0,17,0.00
2020-09-01 10:00:00,4,28,0,28,0.00
2020-09-01 10:00:00,9,22,0,22,0.00
...,...,...,...,...,...
2021-07-30 14:00:00,14,1,2,3,0.67
2021-07-30 14:00:00,18,3,0,3,0.00
2021-07-30 14:00:00,21,1,0,1,0.00


## 5 - Análise Exploratória